In [29]:
import numpy as np
import scipy.optimize as opt

import pandas as pd

import pyretina

from pyretina.optimize import multistart_until
from pyretina.rogozhnikov_curve2 import RogozhnikovCurve2

%matplotlib inline

DATA_PATH = "./data/MC/"

In [12]:
from ipyparallel import Client

rc = Client()

lview = rc.load_balanced_view()

In [13]:
import os
import re

events = list()
r = re.compile("\\d+[_]\\d+")

event_names = list(set([ r.findall(item)[0] for item in os.listdir(DATA_PATH) if len(r.findall(item)) > 0 ]))

In [14]:
%%px

import sys

sys.path.append("./pyretina")

In [41]:
%%px

import pandas as pd

import pyretina
from pyretina.optimize import multistart_until
from pyretina.rogozhnikov_curve2 import RogozhnikovCurve2

DATA_PATH = "/home/mborisyak/notebook/pyretina/data/MC/"
    
bounds = np.array([
      [0, 2], # ax
      [0, 2], # bx
      [0, 2], # ay
      [0, 2], # by
      [-6000, 5000], # x0
      [-6000, 5000], # y0
      [5000, 6000],  # z0
      [-0.1, 0.1] # gamma
])

In [ ]:
def fit_event(item):
   

        
        results = list()
        
        for ei in range(tx.shape[0]):
            f, f_jac, f_hess = rogozhnikov_curve2.RogozhnikovCurve2(
                tx[ei, :], ty[ei, :], tz[ei, :], spline=True, spline_n=30).mse()

            res = multistart_until(bounds, 1.0e+2, f, n_attempts = 250, f_jac=f_jac,
                method="BFGS", strict=False, solver_options={ "maxiter" : 100, "gtol" : 1.0e-5}
            )

            results.append(res)

        return (item, results)
    except Exception as e:
        print e
        return (item, [])

In [65]:
fitted = lview.map(fit_event, event_names)

In [83]:
print "total", len(event_names)

from time import sleep

while not fitted.ready():
    elapsed = fitted.elapsed
    work_done = float(fitted.progress)
    
    speed = work_done / elapsed
    left = len(event_names) - fitted.progress
    eta = left / speed
    
    print "Done:", fitted.progress, "elapsed:", fitted.elapsed, "speed:", speed, "events / sec"
    print "left:", eta / 3600, "hours"
    import sys
    sys.stdout.flush()
    sleep(60.0)

total 2579
Done: 195 elapsed: 589.342198 speed: 0.330893261938 events / sec
left: 2.00131673381 hours
Done: 212 elapsed: 649.478861 speed: 0.32642960786 events / sec
left: 2.01421679948 hours
Done: 233 elapsed: 709.604265 speed: 0.328357382078 events / sec
left: 1.98462620984 hours
Done: 246 elapsed: 769.732439 speed: 0.319599725612 events / sec
left: 2.02770998728 hours
Done: 267 elapsed: 829.891457 speed: 0.32173650916 events / sec
left: 1.99611235883 hours
Done: 291 elapsed: 890.052841 speed: 0.326957031903 events / sec
left: 1.94385039482 hours
Done: 307 elapsed: 950.21139 speed: 0.323095491733 events / sec
left: 1.95332688712 hours
Done: 324 elapsed: 1010.363263 speed: 0.320680500174 events / sec
left: 1.95331143786 hours
Done: 344 elapsed: 1070.529747 speed: 0.321348285635 events / sec
left: 1.93196404364 hours
Done: 361 elapsed: 1130.650543 speed: 0.319288514103 events / sec
left: 1.9296375657 hours
Done: 375 elapsed: 1190.782327 speed: 0.314926414596 events / sec
left: 1.944016

In [118]:
results = [ r for r in fitted.get() if len(r) == 2 ]

In [134]:
not (np.array())

False

In [145]:
def clean(results):
    for (item, result) in results:
        params = np.zeros((len(result), 8))
        errors = np.array([ opt.fun for opt in result ])
        if not np.any(np.isnan(errors)) and np.all(errors < 100.0):
            for i in xrange(params.shape[0]):
                params[i, :] = result[i].x
            yield (item, params)

In [146]:
cleaned = list(clean(results))

In [150]:
for item, params in cleaned:
    np.savetxt("%s.fitted.csv" % item, params, delimiter = ",")

In [ ]:
def max_error(results):
    for (item, result) in results:
        params = np.zeros((len(result), 8))
        errors = np.array([ opt.fun for opt in result ])
        if not np.any(np.isnan(errors)) and np.all(errors < 100.0):
            for i in xrange(params.shape[0]):
                params[i, :] = result[i].x
            yield (item, params)

In [151]:
len(cleaned)

790